# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.


## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas

import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the `.csv` file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp500_list.csv')
from secret_file import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2624802083999,
 'week52high': 187.4,
 'week52low': 130.82,
 'week52highSplitAdjustOnly': 186.59,
 'week52lowSplitAdjustOnly': 132.46,
 'week52change': 0.06429001235416673,
 'sharesOutstanding': 16965607457,
 'float': 0,
 'avg10Volume': 75690703,
 'avg30Volume': 76209165,
 'day200MovingAvg': 162.46,
 'day50MovingAvg': 145.03,
 'employees': 151663,
 'ttmEPS': 6.32,
 'ttmDividendRate': 0.9156304220317496,
 'dividendYield': 0.005780513270450824,
 'nextDividendDate': '',
 'exDividendDate': '2022-04-25',
 'nextEarningsDate': '2022-07-23',
 'peRatio': 25.85868274410706,
 'beta': 1.295495671641212,
 'maxChangePercent': 61.3801967946991,
 'year5ChangePercent': 3.428937483921202,
 'year2ChangePercent': 0.6292222799604702,
 'year1ChangePercent': 0.07822151848663335,
 'ytdChangePercent': -0.12308672512255929,
 'month6ChangePercent': -0.042192439739649,
 'month3ChangePercent': -0.0676159402067206,
 'month1ChangePercent': 0.14623282575720403,
 'day30ChangeP

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example.

In [4]:
data['year1ChangePercent']

0.07822151848663335

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called `chunks` that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Symbol'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    
for symbol_string in symbol_strings:
    print(symbol_string)

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

MMM,AOS,ABT,ABBV,ABMD,ACN,ATVI,ADM,ADBE,ADP,AAP,AES,AFL,A,APD,AKAM,ALK,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AMD,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,ABC,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ANET,AJG,AIZ,T,ATO,ADSK,AZO,AVB,AVY,BKR,BALL,BAC,BBWI,BAX,BDX,WRB,BRK.B,BBY,BIO,TECH,BIIB,BLK,BK,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,CHRW,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,CNC,CNP,CDAY
CF,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CTXS,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CTVA,COST,CTRA,CCI,CSX,CMI,CVS,DHI,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DISH,DIS,DG,DLTR,D,DPZ,DOV,DOW,DTE,DUK,DRE,DD,DXC,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,LLY,EMR,ENPH,ETR,EOG,EPAM,EFX,EQIX,EQR,ESS,EL,ETSY,RE,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FAST,FRT,FDX,FITB,FRC,FE,FIS
FISV,FLT,FMC,F,FTNT,FTV,FBHS,FOXA,FOX,BEN,FCX,GRMN,IT,GNRC,GD,GE,GIS,GM,GPC,GILD,GL,GPN,GS,HAL,HIG,HAS,HCA,PEAK,HSIC,HSY,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HWM,HPQ,HUM

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
df = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        df = df.append(
        pd.Series(
        [symbol,data[symbol]['price'],data[symbol]['stats']['year1ChangePercent'],'N/A'],index = my_columns),ignore_index = True)
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,MMM,135.74,-0.347830,N/A
1,AOS,60.06,-0.159102,N/A
2,ABT,111.87,-0.056977,N/A
3,ABBV,152.19,0.320047,N/A
4,ABMD,288.06,-0.119233,N/A
...,...,...,...,...
498,YUM,123.26,0.047752,N/A
499,ZBRA,337.98,-0.386083,N/A
500,ZBH,107.91,-0.326066,N/A
501,ZION,54.42,0.027399,N/A


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.


In [7]:
df.sort_values('One-Year Price Return',ascending=False,inplace=True)
df = df[:50]
df.reset_index(drop=True,inplace=True)
df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,OXY,61.950,1.348527,N/A
1,DVN,56.620,1.297039,N/A
2,MRO,22.624,0.896216,N/A
3,APA,33.240,0.837846,N/A
4,CF,87.000,0.830934,N/A
5,CTRA,28.453,0.772413,N/A
6,DLTR,172.480,0.748614,N/A
7,MCK,330.149,0.693128,N/A
8,VLO,104.350,0.691333,N/A
9,MPC,88.140,0.674630,N/A


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except: 
        print('That is not a number! \nPlease try again:')
        portfolio_size = input('Enter the size of your portfolio:')

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio:1000000
1000000


In [9]:
position_size = float(portfolio_size)/len(df.index)
for i in range(0,len(df)):
    df.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/df.loc[i,'Price'])

df

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,OXY,61.950,1.348527,322
1,DVN,56.620,1.297039,353
2,MRO,22.624,0.896216,884
3,APA,33.240,0.837846,601
4,CF,87.000,0.830934,229
5,CTRA,28.453,0.772413,702
6,DLTR,172.480,0.748614,115
7,MCK,330.149,0.693128,60
8,VLO,104.350,0.691333,191
9,MPC,88.140,0.674630,226


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

* High-quality momentum stocks show "slow and steady" outperformance over long periods of time
* Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of: 

* 1-month price returns
* 3-month price returns
* 6-month price returns
* 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation `hqm` often. It stands for `high-quality momentum`.

In [10]:
hqm_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy',
    'One-Year Price Return', 
    'One-Year Return Percentile',
    'Six-Month Price Return',
    'Six-Month Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'HQM Score'
]

hqm_df = pd.DataFrame(columns=hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=price,stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_df = hqm_df.append(
        pd.Series(
        [    
            symbol, 
            data[symbol]['price'], 
            'N/A',
            data[symbol]['stats']['year1ChangePercent'], 
            'N/A',
            data[symbol]['stats']['month6ChangePercent'],
            'N/A',
            data[symbol]['stats']['month3ChangePercent'],
            'N/A',
            data[symbol]['stats']['month1ChangePercent'],
            'N/A',
            'N/A'
        ],index = hqm_columns),ignore_index = True)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,138.570,N/A,-0.340665,N/A,-0.224425,N/A,-0.118522,N/A,0.031217,N/A,N/A
1,AOS,58.732,N/A,-0.160219,N/A,-0.228608,N/A,-0.107097,N/A,0.118103,N/A,N/A
2,ABT,108.820,N/A,-0.056348,N/A,-0.118330,N/A,-0.103495,N/A,0.067314,N/A,N/A
3,ABBV,152.680,N/A,0.328733,N/A,0.141092,N/A,-0.061415,N/A,0.039523,N/A,N/A
4,ABMD,281.630,N/A,-0.118364,N/A,-0.067405,N/A,-0.098801,N/A,0.229355,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,122.510,N/A,0.049874,N/A,-0.022877,N/A,-0.029673,N/A,0.078690,N/A,N/A
499,ZBRA,338.890,N/A,-0.392061,N/A,-0.328863,N/A,-0.151427,N/A,0.125993,N/A,N/A
500,ZBH,108.020,N/A,-0.316768,N/A,-0.113435,N/A,-0.179023,N/A,0.064685,N/A,N/A
501,ZION,52.410,N/A,0.027610,N/A,-0.146458,N/A,-0.149977,N/A,0.000567,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* `One-Year Price Return`
* `Six-Month Price Return`
* `Three-Month Price Return`
* `One-Month Price Return`

Here's how we'll do this:

In [11]:
time_periods = ['One-Year','Six-Month','Three-Month','One-Month']

for row in hqm_df.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_df.loc[row,percentile_col] = score(hqm_df[change_col],hqm_df.loc[row,change_col])/100
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,138.570,N/A,-0.340665,0.089463,-0.224425,0.145129,-0.118522,0.395626,0.031217,0.284294,N/A
1,AOS,58.732,N/A,-0.160219,0.312127,-0.228608,0.139165,-0.107097,0.449304,0.118103,0.777336,N/A
2,ABT,108.820,N/A,-0.056348,0.520875,-0.118330,0.397614,-0.103495,0.465209,0.067314,0.518887,N/A
3,ABBV,152.680,N/A,0.328733,0.946322,0.141092,0.918489,-0.061415,0.638171,0.039523,0.347913,N/A
4,ABMD,281.630,N/A,-0.118364,0.403579,-0.067405,0.552684,-0.098801,0.491054,0.229355,0.978131,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,122.510,N/A,0.049874,0.709742,-0.022877,0.677932,-0.029673,0.747515,0.078690,0.580517,N/A
499,ZBRA,338.890,N/A,-0.392061,0.065606,-0.328863,0.04175,-0.151427,0.26839,0.125993,0.819085,N/A
500,ZBH,108.020,N/A,-0.316768,0.109344,-0.113435,0.415507,-0.179023,0.188867,0.064685,0.50497,N/A
501,ZION,52.410,N/A,0.027610,0.673956,-0.146458,0.304175,-0.149977,0.272366,0.000567,0.15507,N/A


## Calculating the HQM Score

We'll now calculate our `HQM Score`, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The `HQM Score` will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the `mean` function from Python's built-in `statistics` module.

In [12]:
from statistics import mean

for row in hqm_df.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_df.loc[row, f'{time_period} Return Percentile'])
    hqm_df.loc[row, 'HQM Score'] = mean(momentum_percentiles)
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,MMM,138.570,N/A,-0.340665,0.089463,-0.224425,0.145129,-0.118522,0.395626,0.031217,0.284294,0.228628
1,AOS,58.732,N/A,-0.160219,0.312127,-0.228608,0.139165,-0.107097,0.449304,0.118103,0.777336,0.419483
2,ABT,108.820,N/A,-0.056348,0.520875,-0.118330,0.397614,-0.103495,0.465209,0.067314,0.518887,0.475646
3,ABBV,152.680,N/A,0.328733,0.946322,0.141092,0.918489,-0.061415,0.638171,0.039523,0.347913,0.712724
4,ABMD,281.630,N/A,-0.118364,0.403579,-0.067405,0.552684,-0.098801,0.491054,0.229355,0.978131,0.606362
...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,122.510,N/A,0.049874,0.709742,-0.022877,0.677932,-0.029673,0.747515,0.078690,0.580517,0.678926
499,ZBRA,338.890,N/A,-0.392061,0.065606,-0.328863,0.04175,-0.151427,0.26839,0.125993,0.819085,0.298708
500,ZBH,108.020,N/A,-0.316768,0.109344,-0.113435,0.415507,-0.179023,0.188867,0.064685,0.50497,0.304672
501,ZION,52.410,N/A,0.027610,0.673956,-0.146458,0.304175,-0.149977,0.272366,0.000567,0.15507,0.351392


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the `HQM Score` column and dropping all but the top 50 entries.

In [13]:
hqm_df.sort_values('HQM Score', ascending=False, inplace=True)
hqm_df = hqm_df[:50]
hqm_df.reset_index(drop=True,inplace=True )
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ON,59.940,N/A,0.684994,0.984095,0.121597,0.908549,0.149702,0.988072,0.182007,0.95825,0.959742
1,ENPH,213.190,N/A,0.229048,0.916501,0.754085,0.998012,0.409915,1.0,0.139395,0.87674,0.947813
2,PWR,132.800,N/A,0.511973,0.970179,0.356452,0.988072,0.065581,0.946322,0.119533,0.789264,0.923459
3,CDNS,169.150,N/A,0.207929,0.890656,0.143081,0.920477,0.114568,0.980119,0.144112,0.890656,0.920477
4,LLY,330.320,N/A,0.401975,0.95825,0.365459,0.99006,0.133494,0.982107,0.103665,0.715706,0.911531
5,SNPS,344.160,N/A,0.217658,0.900596,0.119062,0.900596,0.146373,0.986083,0.122412,0.803181,0.897614
6,CNC,91.880,N/A,0.241557,0.920477,0.177000,0.942346,0.030754,0.898608,0.121104,0.801193,0.890656
7,VICI,34.000,N/A,0.083736,0.759443,0.249102,0.968191,0.109964,0.976143,0.119859,0.791252,0.873757
8,DLTR,177.592,N/A,0.740102,0.988072,0.375831,0.992048,-0.008206,0.817097,0.099115,0.681909,0.869781
9,OXY,63.530,N/A,1.302553,0.998012,0.871673,1.0,0.050643,0.932406,0.071460,0.54672,0.869284


## Calculating the Number of Shares to Buy

We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a `for` loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the size of your portfolio:1000000


In [15]:
position_size = float(portfolio_size)/len(hqm_df.index)
for i in hqm_df.index:
    hqm_df.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_df.loc[i,'Price'])
    
hqm_df

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,ON,59.940,333,0.684994,0.984095,0.121597,0.908549,0.149702,0.988072,0.182007,0.95825,0.959742
1,ENPH,213.190,93,0.229048,0.916501,0.754085,0.998012,0.409915,1.0,0.139395,0.87674,0.947813
2,PWR,132.800,150,0.511973,0.970179,0.356452,0.988072,0.065581,0.946322,0.119533,0.789264,0.923459
3,CDNS,169.150,118,0.207929,0.890656,0.143081,0.920477,0.114568,0.980119,0.144112,0.890656,0.920477
4,LLY,330.320,60,0.401975,0.95825,0.365459,0.99006,0.133494,0.982107,0.103665,0.715706,0.911531
5,SNPS,344.160,58,0.217658,0.900596,0.119062,0.900596,0.146373,0.986083,0.122412,0.803181,0.897614
6,CNC,91.880,217,0.241557,0.920477,0.177000,0.942346,0.030754,0.898608,0.121104,0.801193,0.890656
7,VICI,34.000,588,0.083736,0.759443,0.249102,0.968191,0.109964,0.976143,0.119859,0.791252,0.873757
8,DLTR,177.592,112,0.740102,0.988072,0.375831,0.992048,-0.008206,0.817097,0.099115,0.681909,0.869781
9,OXY,63.530,314,1.302553,0.998012,0.871673,1.0,0.050643,0.932406,0.071460,0.54672,0.869284


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('Momentum_strategy.xlsx', engine='xlsxwriter')
hqm_df.to_excel(writer, sheet_name="Momentum Strategy",index=False )

## Creating the Formats We'll Need For Our .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_format = hqm_columns = {
    'A':['Ticker', string_template],
    'B':['Price', dollar_template],
    'C':['Number of Shares to Buy',integer_template],
    'D':['One-Year Price Return', percent_template],
    'E':['One-Year Return Percentile',percent_template],
    'F':['Six-Month Price Return',percent_template],
    'G':['Six-Month Return Percentile',percent_template],
    'H':['Three-Month Price Return',percent_template],
    'I':['Three-Month Return Percentile',percent_template],
    'J':['One-Month Price Return',percent_template],
    'K':['One-Month Return Percentile',percent_template],
    'L':['HQM Score',percent_template]
}

for column in column_format.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}',25,column_format[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1',column_format[column][0],column_format[column][1])
    


## Saving Our Excel Output

As before, saving our Excel output is very easy:

In [19]:
writer.save()